<a href="https://colab.research.google.com/github/KA1exe1AK/AI_course/blob/main/%D0%9B%D0%B0%D0%B1%D0%B03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>